<p align="center">
  <img src="image.png" alt="Logo du projet" style="width: 150px; height: auto; border-radius: 8px; box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);">
</p>

<h1 align="center" style="color: #FFA500; font-size: 2.5em; font-weight: bold;">Projet INS_PHAS</h1>

<p align="center" style="color: #4CAF50; font-size: 1.2em;">
  Analyse des offres d’emploi au moyen des données scrapées.
</p>

<p align="center" style="color: #4CAF50; font-size: 1.1em;">
  <strong>Auteur :</strong> DOUMBIA ABDOULAYE (<a href="mailto:abdoulaye.doumbi19@inphb.ci" style="color: #FFA500;">abdoulaye.doumbi19@inphb.ci</a>)
</p>

<p align="center" style="color: #4CAF50; font-size: 1.1em;">
  Ce matériel est soumis aux termes et conditions de la licence <a href="https://creativecommons.org/licenses/by-nc-sa/4.0/" style="color: #FFA500;">Creative Commons CC BY-NC-SA 4.0</a>. L'utilisation gratuite est autorisée à des fins non commerciales.
</p>


# Educarriere

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_text(element, class_name=None, style=None, text_contains=None):
    if element:
        tag = element.find(class_=class_name, style=style, text=text_contains)
        return tag.text.strip() if tag else ""
    else:
        return ""

def clean_text(text):
    if text is not None:
        cleaned_text = text.replace('D\x92', ' ').replace('d\x92', ' ').replace('\x92', ' ').replace('\r\n', '').replace('\xa0', '')
        return cleaned_text.strip() if cleaned_text else None
    else:
        return None

def extract_date(element, text_contains):
    date_elements = element.find_all('a', class_='text')
    date = next((e.find('span', style='color:#FF0000;').text.strip() for e in date_elements if text_contains in e.text), "")
    return date

def scrape_emploi_ci(url):
    try:
        response = requests.get(url, timeout=200)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Erreur de connexion à {url} : {e}")
        return pd.DataFrame()

    soup = BeautifulSoup(response.text, 'html.parser')

    job_description_wrappers = soup.find_all('div', class_='box row')

    data_list = []

    for wrapper in job_description_wrappers:
        h4_tag = wrapper.find('h4')
        poste = extract_text(h4_tag)

        entry_title_tag = wrapper.find('p', class_='entry-title')
        sous_titre = extract_text(entry_title_tag)

        a_text_tag = wrapper.find('a', class_='text')
        code = extract_text(a_text_tag, style='color:#FF0000;')

        date_edition = extract_date(wrapper, "Date d'édition:")
        date_limite = extract_date(wrapper, "Date limite:")

        pays_tag = wrapper.find('a', class_='text')
        pays = pays_tag.find_parent().text.strip().split()[-1] if pays_tag else None

        sous_titre = clean_text(sous_titre)

        data_list.append({
            'Poste': clean_text(poste),
            'Sous_titre': sous_titre,
            'Code': clean_text(code),
            'Date_DEdition': date_edition,
            'Date_limite': date_limite,
            'Pays': clean_text(pays)
        })

    df = pd.DataFrame(data_list)
    return df

# Liste des liens
urls = ["https://emploi.educarriere.ci/nos-offres?page1={}&codes=&mots_cles=&typeemploi1=&niveau1=&anciennete=&typeoffre1=&recruteur=".format(category) for category in range(40)]

# Créer un DataFrame à partir des liens
result_df = pd.concat([scrape_emploi_ci(url) for url in urls], ignore_index=True)

# Supprimer les lignes dont toutes les variables n'ont pas de données
result_df = result_df.dropna(how='any').reset_index(drop=True)

# Afficher le DataFrame
result_df

Erreur de connexion à https://emploi.educarriere.ci/nos-offres?page1=3&codes=&mots_cles=&typeemploi1=&niveau1=&anciennete=&typeoffre1=&recruteur= : HTTPSConnectionPool(host='emploi.educarriere.ci', port=443): Read timed out.


,Poste,Sous_titre,Code,Date_DEdition,Date_limite,Pays
0,FORMATEUR EN FINANCES COMPTABILITE,SHINE CORPORATErecruteFORMATEUR EN FINANCES CO...,109494,02/02/2024,12/02/2024,Abidjan
1,MAGASINIER,CONCEPTEUR ET CONSTRUCTEUR DE BATIMENTS GROUPr...,109493,02/02/2024,26/02/2024,Abidjan
2,RESEAU INFORMATIQUE ET TELECOMMUNICATION,MoonvisionsrecruteRESEAU INFORMATIQUE ET TELEC...,109492,02/02/2024,08/02/2024,8EM...
3,UN.E HOMME/FEMME DE MENAGE,MEDECINS DU MONDErecruteUN.E HOMME/FEMME DE ME...,109491,02/02/2024,07/02/2024,SAN-PEDRO
4,01 CHARGE DE RECRUTEMENT,CI-HBSrecrute01 CHARGE DE RECRUTEMENTDescripti...,109490,02/02/2024,16/02/2024,d'I...
...,...,...,...,...,...,...
521,STAGE DE SOUTENANCE (RH-COM; GESCOM; FCGE; RIT...,Description du posteStage de soutenanceNous of...,95492,14/02/2023,27/03/2024,...
522,ESTHÉTICIENNE - PROTHÉSISTE ONGULAIRE (H/F),Description du posteNous recherchons des esthé...,94035,16/01/2023,23/04/2025,...
523,CONSULTANT FORMATEUR EN DROIT,"Description du posteNous, cabinet de formation...",86696,27/07/2022,02/03/2024,ABIDJAN
524,CONSULTANT FORMATEUR SPECIALISE,"Description du posteNous, cabinet de formation...",86695,27/07/2022,02/03/2024,ABIDJAN


In [ ]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

# Liste des liens
urls = ["https://emploi.educarriere.ci/nos-offres?page1={}&codes=&mots_cles=&typeemploi1=&niveau1=&anciennete=&typeoffre1=&recruteur=".format(category) for category in range(40)]

# En-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
options.add_argument("--disable-gpu")  # Désactiver l'accélération GPU en mode headless
chrome_driver_path = "C:\\Users\\Dell\\Desktop\\offre_d_emploi\\chromedriver-win32\\chromedriver.exe"
options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"  # Remplacez par l'emplacement réel de votre Chrome binary
options.add_argument(f"webdriver.chrome.driver={chrome_driver_path}")
driver = webdriver.Chrome(options=options)

# Liste pour stocker les détails de chaque emploi
all_job_details = []

# Parcourir les liens
for url in urls:
    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    offres = soup.find_all('div', class_='box row')

    # Parcourir les offres d'emploi sur la page principale
    for offre in offres:
        # Trouver la balise <h4> dans la structure HTML pour extraire le lien
        offre_link_tag = offre.find('h4')

        # Vérifier si la balise <h4> a été trouvée
        if offre_link_tag:
            # Extraire le lien de l'attribut 'href'
            offre_link = offre_link_tag.find('a')['href']
            all_job_details.append({'Offre_Link': offre_link})

# Fermer le pilote Selenium à la fin
driver.quit()

# Concaténer tous les détails des emplois en un seul DataFrame
if all_job_details:
    all_job_details_df = pd.DataFrame(all_job_details)
    # Afficher le DataFrame
    #print(all_job_details_df)
else:
    print("Aucun détail d'offre d'emploi trouvé.")


Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event): operation timed out


In [ ]:
all_job_details_df

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib3
import time
from selenium import webdriver

# Désactiver les avertissements liés aux requêtes non sécurisées
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# (votre fonction extract_job_information reste inchangée)
def extract_job_information(soup):
    # Extraction des informations de l'offre d'emploi
    poste = soup.select_one('li.list-group-item:-soup-contains("Poste")').strong.next_sibling.strip()
    type_offre = soup.select_one('li.list-group-item:-soup-contains("Type d\'offre")').strong.next_sibling.strip()
    metiers = soup.select_one('li.list-group-item:-soup-contains("Métier(s):")').strong.next_sibling.strip()
    niveaux = soup.select_one('li.list-group-item:-soup-contains("Niveau(x):")').strong.next_sibling.strip()
    experience = soup.select_one('li.list-group-item:-soup-contains("Expérience:")').strong.next_sibling.strip()
    lieu = soup.select_one('li.list-group-item:-soup-contains("Lieu:")').strong.next_sibling.strip()
    
    # Extraction des dates de publication et de limite
    date_publication = soup.find('strong', string='Date de publication:').find_next('span').text.strip()
    date_limite = soup.find('strong', string='Date limite:').find_next('span').text.strip()
    
    # description = soup.select_one('div.text-col.post.small-post.col-md-9.col-xs-12 ul.list-group').text.strip()
    description = soup.select_one('div.entry-content').text.strip()

    return {
        "Poste": [poste],
        "Type d'offre": [type_offre],
        "Métier(s)": [metiers],
        "Niveau(x)": [niveaux],
        "Expérience": [experience],
        "Lieu": [lieu],
        "Date de publication": [date_publication],
        "Date limite": [date_limite],
        "Description": [description]
    }

# Liste des liens
urls = ["https://emploi.educarriere.ci/nos-offres?page1={}&codes=&mots_cles=&typeemploi1=&niveau1=&anciennete=&typeoffre1=&recruteur=".format(category) for category in range(40)]

# En-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
options.add_argument("--disable-gpu")  # Désactiver l'accélération GPU en mode headless
chrome_driver_path = "C:\\Users\\ngora\\OneDrive\\Bureau\\INS_DATA\\chromedriver_win32\\chromedriver.exe"
options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"  # Remplacez par l'emplacement réel de votre Chrome binary
options.add_argument(f"webdriver.chrome.driver={chrome_driver_path}")
driver = webdriver.Chrome(options=options)

# Liste pour stocker les DataFrames
dfs = []

# Boucle sur chaque URL
for url in urls:
    try:
        # Envoyer une requête GET au site avec un délai de 120 secondes
        req = requests.get(url, headers=headers)
        soup = BeautifulSoup(req.text, 'html.parser')
        time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

        offres = soup.find_all('div', class_='box row')

        # Parcourir les offres d'emploi sur la page principale
        for offre in offres:
            offre_link_tag = offre.find('h4')
            if offre_link_tag:
                offre_link = offre_link_tag.find('a')['href']

                # Ajouter l'URL à la liste des informations
                all_job_details = {'Offre_Link': offre_link, 'URL': url}

                try:
                    # Extraire les informations sur l'emploi
                    job_info = extract_job_information(soup)

                    # Ajouter l'URL à la liste des informations
                    job_info['URL'] = [url]

                    # Créer un DataFrame
                    df = pd.DataFrame(job_info)

                    # Ajouter le DataFrame à la liste
                    dfs.append(df)
                except Exception as e:
                    print(f"An error occurred while extracting job information for URL {url}: {e}")
                    # En cas d'erreur, créer un DataFrame avec l'URL et une colonne vide
                    df = pd.DataFrame({"URL": [url]})
                    dfs.append(df)
    except requests.exceptions.Timeout:
        print(f"Timeout lors de la requête pour l'URL {url}")
        # En cas de timeout, créer un DataFrame avec l'URL et une colonne vide
        df = pd.DataFrame({"URL": [url]})
        dfs.append(df)
    except requests.exceptions.RequestException as e:
        print(f"Une erreur s'est produite lors de la requête pour l'URL {url}: {e}")
        # En cas d'autre exception, créer un DataFrame avec l'URL et une colonne vide
        df = pd.DataFrame({"URL": [url]})
        dfs.append(df)

    # Ajouter un délai de 5 secondes entre les requêtes pour éviter d'être bloqué
    time.sleep(5)

# Fermer le pilote Selenium à la fin
driver.quit()

# Concaténer tous les DataFrames en un seul DataFrame
df_Educarriere = pd.concat(dfs, ignore_index=True)
df_Educarriere

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib3
import time

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# (votre fonction extract_job_information reste inchangée)
def extract_job_information(soup):
    # Extraction des informations de l'offre d'emploi
    poste = soup.select_one('li.list-group-item:-soup-contains("Poste")').strong.next_sibling.strip()
    type_offre = soup.select_one('li.list-group-item:-soup-contains("Type d\'offre")').strong.next_sibling.strip()
    metiers = soup.select_one('li.list-group-item:-soup-contains("Métier(s):")').strong.next_sibling.strip()
    niveaux = soup.select_one('li.list-group-item:-soup-contains("Niveau(x):")').strong.next_sibling.strip()
    experience = soup.select_one('li.list-group-item:-soup-contains("Expérience:")').strong.next_sibling.strip()
    lieu = soup.select_one('li.list-group-item:-soup-contains("Lieu:")').strong.next_sibling.strip()
    
    # Extraction des dates de publication et de limite
    date_publication = soup.find('strong', string='Date de publication:').find_next('span').text.strip()
    date_limite = soup.find('strong', string='Date limite:').find_next('span').text.strip()
    
   # description = soup.select_one('div.text-col.post.small-post.col-md-9.col-xs-12 ul.list-group').text.strip()
    description = soup.select_one('div.entry-content').text.strip()

    return {
        "Poste": [poste],
        "Type d'offre": [type_offre],
        "Métier(s)": [metiers],
        "Niveau(x)": [niveaux],
        "Expérience": [experience],
        "Lieu": [lieu],
        "Date de publication": [date_publication],
        "Date limite": [date_limite],
        "Description": [description]
    }

# Liste des URLs à scraper
urls = list(all_job_details_df['Offre_Link'])

# Liste pour stocker les DataFrames
dfs = []

# Boucle sur chaque URL
for url in urls:
    try:
        # Envoyer une requête GET au site avec un délai de 120 secondes
        response = requests.get(url, headers=headers, verify=True, timeout=120)

        # Vérifier si la requête a réussi (statut 200)
        if response.status_code == 200:
            # Analyser le contenu de la page avec BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')

            try:
                # Extraire les informations sur l'emploi
                job_info = extract_job_information(soup)

                # Créer un DataFrame
                df = pd.DataFrame(job_info)

                # Ajouter le DataFrame à la liste
                dfs.append(df)
            except Exception as e:
                print(f"An error occurred while extracting job information: {e}")
        else:
            print(f"Échec de la requête pour l'URL {url}. Statut : {response.status_code}")

    except requests.exceptions.Timeout:
        print(f"Timeout lors de la requête pour l'URL {url}")
    except requests.exceptions.RequestException as e:
        print(f"Une erreur s'est produite lors de la requête pour l'URL {url}: {e}")

    # Ajouter un délai de 5 secondes entre les requêtes pour éviter d'être bloqué
    time.sleep(5)

# Concaténer tous les DataFrames en un seul DataFrame
df_Educarriere = pd.concat(dfs, ignore_index=True)
df_Educarriere


In [ ]:
# Ajouter les listes existantes en tant que colonnes au DataFrame
df_Educarriere['Poste'] = list(result_df['Poste'])
df_Educarriere['Sous_titre'] = list(result_df['Sous_titre'])
df_Educarriere['Code'] = list(result_df['Code'])
df_Educarriere['Date_DEdition'] = list(result_df['Date_DEdition'])
df_Educarriere['Date_limite'] = list(result_df['Date_limite'])
df_Educarriere['Pays'] = list(result_df['Pays'])



# Réorganiser les colonnes selon vos besoins
df_Educarriere 

In [ ]:


# Exportez le DataFrame en fichier CSV
df_Educarriere.to_csv("df_Educarriere.csv", index=False)

# NOVOJOB

In [ ]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

# Liste des liens pour chaque catégorie
categories = [
    "toutes les offres d'emploi",
    "juridique,fiscal,audit,conseil",
    "administrations,moyens généraux",
    "assistanat,secrétariat",
    "metiers banque et assurances",
    "RH,personnel,formation",
    "education,enseignement",
    "direction générale,direction d'unité",
    "vente,televente,assistanat",
    "commercial,technico commercial,service client",
    "responsable commercial,grands comptes",
    "créatio, design",
    "marketing, communication",
    "journalisme,médias,traduction",
    "informatique,systèmes d'information,internet",
    "télécommunication,réseaux",
    "chantier,métiers BTP,architecture",
    "ingénierie,etudes,projet,R&D",
    "logistique,achat,stock,transport",
    "production,méthode,industrie",
    "maintenance,entretien",
    "Qualité,sécurité,Environnement",
    "Santé,Médical,Pharmacie",
    "Hotellerie,Tourisme,Restauration, Loisirs",
    "Ouvriers qualifiés, Chauffeur",
    "autre",
    "Métiers de l'agriculture"
]

base_url = "https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q="
category_links = [f"{base_url}{'+'.join(category.split(','))}" for category in categories]

intitules_list = []
entreprises_list = []
pays_list = []
dates_list = []
lien_list = []
niveau_list = []
experience_list = []

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    if 'finance' in category_link:
        offres = soup.find_all('h2', class_='ellipsis row-fluid')
        entreprises = soup.find_all('h6', class_='ellipsis')
        niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')
    else:
        offres = soup.find_all('h2', class_='ellipsis row-fluid')
        entreprises = soup.find_all('h6', class_='ellipsis')
        niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')

    for offre, entreprise, niveau in zip(offres, entreprises, niveaux):
        bloc_bottom = offre.find_next('div', class_='bloc-bottom')
        intitules_list.append(offre.get_text().strip())
        entreprises_list.append(entreprise.get_text().strip())
        lien_list.append(category_links.index(category_link))

        # Les informations (pays, date, niveau, expérience) sont contenues dans la même span, nous devons les séparer
        pays_info = bloc_bottom.find('i', class_='fa fa-map-marker icon-left')
        pays = pays_info.find_parent().text.strip() if pays_info else None
        pays_list.append(pays)

        date_info = bloc_bottom.find('i', class_='fa fa-clock-o icon-left')
        date = date_info.find_parent().text.strip() if date_info else None
        dates_list.append(date)

        # Ajout des colonnes pour le niveau du poste et l'expérience demandée
        niveau_info = niveau.find('i', class_='fa fa-bookmark icon-left')
        niveau_text = niveau_info.find_parent().text.strip() if niveau_info else None

        # Utiliser une expression régulière pour extraire les informations de niveau et d'expérience
        match = re.match(r'(.+) \((.+)\)', niveau_text)

        if match:
            niveau_col, experience_col = match.groups()
        else:
            niveau_col, experience_col = None, None

        niveau_list.append(niveau_col)
        experience_list.append(experience_col)



In [ ]:
df_offers = pd.DataFrame({
    'Intitule': intitules_list,
    'Entreprise': entreprises_list,
    'Pays': pays_list,
    'Date': dates_list,
    'Niveau': niveau_list,
    'Experience_lettre': experience_list,
    'Lien': lien_list
})
df_offers

In [ ]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

# Liste des liens pour chaque catégorie
categories = [
    "toutes les offres d'emploi",
    "juridique,fiscal,audit,conseil",
    "administrations,moyens généraux",
    "assistanat,secrétariat",
    "metiers banque et assurances",
    "RH,personnel,formation",
    "education,enseignement",
    "direction générale,direction d'unité",
    "vente,televente,assistanat",
    "commercial,technico commercial,service client",
    "responsable commercial,grands comptes",
    "créatio, design",
    "marketing, communication",
    "journalisme,médias,traduction",
    "informatique,systèmes d'information,internet",
    "télécommunication,réseaux",
    "chantier,métiers BTP,architecture",
    "ingénierie,etudes,projet,R&D",
    "logistique,achat,stock,transport",
    "production,méthode,industrie",
    "maintenance,entretien",
    "Qualité,sécurité,Environnement",
    "Santé,Médical,Pharmacie",
    "Hotellerie,Tourisme,Restauration, Loisirs",
    "Ouvriers qualifiés, Chauffeur",
    "autre",
    "Métiers de l'agriculture"
]

base_url = "https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q="
category_links = [f"{base_url}{'+'.join(category.split(','))}" for category in categories]

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Initialiser le pilote Selenium
driver = webdriver.Chrome()

# Liste pour stocker les détails de chaque emploi
all_job_lien = []

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    offres = soup.find_all('div', class_='row-fluid job-details pointer')

    # Parcourir les offres d'emploi sur la page principale
    for offre in offres:
        # Trouver la balise <a> dans la structure HTML pour extraire le lien
        offre_link_tag = offre.find('a')

        # Vérifier si la balise <a> a été trouvée
        if offre_link_tag:
            # Extraire le lien de l'attribut 'href'
            offre_link = offre_link_tag['href']

            
            # Ajouter les détails de l'emploi à la liste
            all_job_lien.append(offre_link)

# Fermer le pilote Selenium à la fin
driver.quit()

# Convertir les détails des offres d'emploi en DataFrame
# Convertir les détails des offres d'emploi en DataFrame
df_Novojob = pd.DataFrame(all_job_lien, columns=['Offre_Link'])
df_Novojob.head()


In [ ]:

import requests
import pandas as pd

# Example usage for multiple job URLs
job_urls = list(df_Novojob['Offre_Link'])

# Extract details for each job URL
all_job_details = []
for url in job_urls:
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')

    job_details = {}
    
    # Ajouter le lien
    job_details["Offre_Link"]=url
    # Extracting job details
    details_section = soup.find('ul', class_='text-small')
    if details_section:
        for li in details_section.find_all('li', class_='row-fluid'):
            key = li.find('span', class_='span4').text.strip()
            value = li.find('span', class_='span8').text.strip()
            job_details[key] = value

    # Extracting the provided text
    description_section = soup.find('div', class_='spaced details-description')
    if description_section:
        provided_text = description_section.text.strip()
        job_details['Provided Text'] = provided_text

    all_job_details.append(job_details)

# Create a DataFrame
df_offers = pd.DataFrame(all_job_details)
df_offers

In [ ]:

import requests
import pandas as pd

# Example usage for multiple job URLs
job_urls = list(df_Novojob['Offre_Link'])

# Extract details for each job URL
all_job_details = []
for url in job_urls:
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')

    job_details = {}

    # Extracting job details
    details_section = soup.find('ul', class_='text-small')
    if details_section:
        for li in details_section.find_all('li', class_='row-fluid'):
            key = li.find('span', class_='span4').text.strip()
            value = li.find('span', class_='span8').text.strip()
            job_details[key] = value

    # Extracting the provided text
    description_section = soup.find('div', class_='spaced details-description')
    if description_section:
        provided_text = description_section.text.strip()
        job_details['Provided Text'] = provided_text

    all_job_details.append(job_details)

# Create a DataFrame
df_offers = pd.DataFrame(all_job_details)
df_offers

In [ ]:
# Ajouter les listes existantes en tant que colonnes au DataFrame
df_offers['Intitule'] = intitules_list
df_offers['Entreprise'] = entreprises_list
df_offers['Pays'] = pays_list
df_offers['Date'] = dates_list
df_offers['Niveau'] = niveau_list
df_offers['Experience_lettre'] = experience_list
df_offers['Lien'] = lien_list

# Réorganiser les colonnes selon vos besoins

df_offers

In [ ]:

# Réorganiser les colonnes selon vos besoins
df_offers = df_offers[[
    'Intitule', 'Entreprise', 'Pays', 'Date', 'Niveau', 'Experience_lettre',
    'Lien', "Lieu de travail", "Date d'expiration", 'Niveau de poste', "Secteur d'activité", "Niveau d'étude (diplome)",
    "Nombre de postes", "Type de contrat", "Provided Text", "Nom de l'entreprise"
]]


df_offers


In [ ]:
# Exportez le DataFrame en fichier CSV
df_offers.to_csv("df_Novojob.csv", index=False)

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Charger les données
# Assurez-vous d'avoir préalablement importé les bibliothèques nécessaires et défini df_offers
# df_offers = pd.read_csv("votre_fichier.csv")  # Remplacez "votre_fichier.csv" par votre fichier de données

# Réorganiser les colonnes selon vos besoins
df_offers = df_offers[[
    'Intitule', 'Entreprise', 'Pays', 'Date', 'Niveau', 'Experience_lettre',
    'Lien', "Lieu de travail", "Date d'expiration", 'Niveau de poste', "Secteur d'activité", "Niveau d'étude (diplome)",
    "Nombre de postes", "Type de contrat", "Provided Text", "Nom de l'entreprise"
]]

# Options pour les filtres
secteurs_activite_options = [{'label': 'Tout sélectionner', 'value': 'all'}] + [{'label': secteur, 'value': secteur} for secteur in df_offers["Secteur d'activité"].unique()]
types_contrat_options = [{'label': 'Tout sélectionner', 'value': 'all'}] + [{'label': contrat, 'value': contrat} for contrat in df_offers["Type de contrat"].unique() if pd.notna(contrat)]

# Palettes de couleurs
secteur_activite_palette = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
entreprise_palette = ['#ffbb78', '#98df8a', '#ff9896', '#c5b0d5', '#c49c94', '#f7b6d2', '#dbdb8d', '#9edae5', '#ff7f0e', '#aec7e8']
type_contrat_palette = ['#ff7f0e', '#1f77b4', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

# Initialiser l'application Dash
app = dash.Dash(__name__)

# Définir la mise en page du tableau de bord
app.layout = html.Div(children=[
    html.H1(children='Tableau de Bord des Offres d\'Emploi'),

    # Filtres interactifs
    html.Div([
        html.Div([
            html.Label('Filtrer par Secteur d\'activité:'),
            dcc.Dropdown(
                id='filtre-secteur-activite',
                options=secteurs_activite_options,
                multi=True,
                placeholder='Sélectionner un secteur d\'activité'
            ),
        ], style={'flex': '1'}),

        html.Div([
            html.Label('Filtrer par Type de contrat:'),
            dcc.Dropdown(
                id='filtre-type-contrat',
                options=types_contrat_options,
                multi=True,
                placeholder='Sélectionner un type de contrat'
            ),
        ], style={'flex': '1'}),

        html.Button('Réinitialiser les filtres', id='bouton-reset', n_clicks=0)
    ], style={'display': 'flex'}),

    # Graphique du nombre d'offres par secteur d'activité
    dcc.Graph(
        id='graph-secteur-activite',
        style={'height': '50vh'},
    ),

    # Graphique du nombre d'offres par entreprise
    dcc.Graph(
        id='graph-entreprise',
        style={'height': '50vh'},
    ),

    # Graphique du type de contrat proposé
    dcc.Graph(
        id='graph-type-contrat',
        style={'height': '50vh'},
    ),
])

# Callback pour mettre à jour les graphiques en fonction des filtres
@app.callback(
    [Output('graph-secteur-activite', 'figure'),
     Output('graph-entreprise', 'figure'),
     Output('graph-type-contrat', 'figure')],
    [Input('filtre-secteur-activite', 'value'),
     Input('filtre-type-contrat', 'value'),
     Input('bouton-reset', 'n_clicks')]
)
def update_graphs(secteurs_activite, types_contrat, n_clicks_reset):
    # Filtrage du DataFrame en fonction des sélections
    df_filtered = df_offers

    # Filtrer par secteur d'activité
    if secteurs_activite and 'all' not in secteurs_activite:
        df_filtered = df_filtered[df_filtered["Secteur d'activité"].isin(secteurs_activite)]

    # Filtrer par type de contrat
    if types_contrat and 'all' not in types_contrat:
        df_filtered = df_filtered[df_filtered["Type de contrat"].isin(types_contrat)]

    # Graphique du nombre d'offres par secteur d'activité
    fig_secteur_activite = px.bar(df_filtered, x="Secteur d'activité", title="Nombre d'offres par secteur d'activité",
                                   color_discrete_sequence=secteur_activite_palette)

    # Graphique du nombre d'offres par entreprise
    fig_entreprise = px.bar(df_filtered, x="Nom de l'entreprise", title="Nombre d'offres par entreprise",
                            color_discrete_sequence=entreprise_palette)

    # Graphique du type de contrat proposé
    fig_type_contrat = px.bar(df_filtered, x="Type de contrat", title="Type de contrat proposé",
                              color_discrete_sequence=type_contrat_palette)

    return fig_secteur_activite, fig_entreprise, fig_type_contrat

# Exécuter l'application Dash
if __name__ == '__main__':
    app.run_server(debug=True)


# 1. Analyse descriptive :


In [ ]:
import pandas as pd

# Afficher les statistiques descriptives
descriptive_stats = df_offers.describe(include='all')
print(descriptive_stats)


# 2. Analyse comparative :

In [ ]:
# Comparaison entre différents secteurs
secteur_comparison = df_offers.groupby("Secteur d'activité").size()
print(secteur_comparison)

# Comparaison entre différentes régions
region_comparison = df_offers.groupby("Pays").size()
print(region_comparison)

# Comparaison entre différents niveaux d'expérience
experience_comparison = df_offers.groupby("Niveau").size()
print(experience_comparison)


# 3. Modélisation :
Si vous souhaitez prédire des tendances futures, vous pourriez envisager une modélisation statistique, par exemple une régression pour estimer les relations entre différentes variables.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Exemple de modélisation simple (n'utilisez que les variables numériques)
X = df_offers[['Nombre de postes']]
y = df_offers['Niveau de poste']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Création et ajustement du modèle de régression linéaire
model = LinearRegression()
model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
predictions = model.predict(X_test)

# Évaluation du modèle
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')


# 4. Interprétation des résultats :
Analysez les résultats de manière critique, identifiez les tendances, les corrélations, et les relations significatives.

# 5. Rapport :
Présentez vos résultats de manière claire dans un rapport en utilisant des graphiques et des tableaux pour illustrer vos conclusions. Vous pouvez utiliser des bibliothèques comme matplotlib ou seaborn pour créer des visualisations.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Exemple de graphique : Nombre d'offres par secteur d'activité
plt.figure(figsize=(12, 6))
sns.countplot(x="Secteur d'activité", data=df_offers)
plt.xticks(rotation=90)
plt.title('Nombre d\'offres par secteur d\'activité')
plt.show()


<p align="center" style="color: #4CAF50; font-size: 1.2em;">
  NLP
</p>

<h1 align="center" style="color: #FFA500; font-size: 2.5em; font-weight: bold;">1. Analyse du Texte de l'Offre :</h1>
<p align="center" style="color: #4CAF50; font-size: 1.2em;">
 Vous pouvez utiliser NLP pour extraire des informations clés des descriptions d'offres d'emploi. Par exemple, pour extraire les compétences requises, les responsabilités, et d'autres informations pertinentes :
</p>




In [ ]:
import spacy

# Charger le modèle NLP
nlp = spacy.load("fr_core_news_sm")

# Appliquer NLP sur le texte de l'offre
texte_offre = df_offers['Provided Text'].iloc[0]  # Utilisez l'indice approprié
doc = nlp(texte_offre)

# Extraire les entités nommées
entites = [(ent.text, ent.label_) for ent in doc.ents]
print("Entités nommées :", entites)

# Extraire les compétences
competences = [token.text for token in doc if "compétence" in token.text.lower()]
print("Compétences requises :", competences)

# D'autres analyses NLP peuvent également être appliquées selon les besoins.


<h1 align="center" style="color: #FFA500; font-size: 2.5em; font-weight: bold;">2. Analyse du Sentiment :</h1>
<p align="center" style="color: #4CAF50; font-size: 1.2em;">
 Vous pouvez utiliser NLP pour évaluer le sentiment général du texte de l'offre, ce qui peut donner une idée de l'atmosphère de travail décrite :
</p>





In [ ]:
from textblob import TextBlob

# Appliquer TextBlob sur le texte de l'offre
analyse_sentiment = TextBlob(texte_offre)

# Extraire le sentiment (positif, négatif, neutre)
sentiment = analyse_sentiment.sentiment.polarity
print("Sentiment de l'offre :", sentiment)


<h1 align="center" style="color: #FFA500; font-size: 2.5em; font-weight: bold;">3. Modélisation de Thèmes :</h1>
<p align="center" style="color: #4CAF50; font-size: 1.2em;">
 L'utilisation de techniques NLP comme le Latent Dirichlet Allocation (LDA) peut vous aider à identifier les thèmes clés présents dans les offres d'emploi :
</p>



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Utiliser CountVectorizer pour convertir le texte en vecteurs
vectorizer = CountVectorizer(stop_words='french')
X = vectorizer.fit_transform(df_offers['Provided Text'])

# Appliquer le modèle LDA
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(X)

# Afficher les mots clés de chaque thème
for i, topic in enumerate(lda.components_):
    mots_cles = [vectorizer.get_feature_names_out()[j] for j in topic.argsort()[-10:]]
    print(f"Thème {i+1} :", mots_cles)


<h1 align="center" style="color: #FFA500; font-size: 2.5em; font-weight: bold;">4. Identification des Relations :</h1>
<p align="center" style="color: #4CAF50; font-size: 1.2em;">
 Vous pouvez utiliser NLP pour identifier des relations entre différentes entités dans le texte, par exemple, entre le lieu de travail et le secteur d'activité.
</p>


In [ ]:
# Exemple d'identification de relations entre le lieu de travail et le secteur d'activité
for ent in doc.ents:
    if ent.label_ == "LOC":
        lieu_travail = ent.text
    elif ent.label_ == "ORG":
        secteur_activite = ent.text

print(f"Lieu de travail : {lieu_travail}, Secteur d'activité : {secteur_activite}")



<p align="center" style="color: #4CAF50; font-size: 1.2em;">
 d'application de NLP sur la colonne "Provided Text" pour extraire des entités nommées, des compétences et effectuer une analyse de sentiment.
</p>


In [ ]:
import pandas as pd
import spacy
from textblob import TextBlob

# Charger le modèle NLP
nlp = spacy.load("fr_core_news_sm")

# Appliquer NLP sur le texte de l'offre (par exemple, prenons la première offre)
texte_offre = df_offers['Provided Text'].iloc[0]
doc = nlp(texte_offre)

# Extraire les entités nommées
entites = [(ent.text, ent.label_) for ent in doc.ents]
print("Entités nommées :", entites)

# Extraire les compétences
competences = [token.text for token in doc if "compétence" in token.text.lower()]
print("Compétences requises :", competences)

# Appliquer TextBlob pour analyser le sentiment
analyse_sentiment = TextBlob(texte_offre)
sentiment = analyse_sentiment.sentiment.polarity
print("Sentiment de l'offre :", sentiment)


# Emploi.ci

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_text(element, tag_name=None):
    tag = element.find(tag_name)
    return tag.text.strip() if tag else ""

def clean_text(text):
    return text.replace('D\x92', ' ').replace('d\x92', ' ').replace('\x92', ' ').replace('\r\n', '').replace('\xa0', '')

def scrape_emploi_ci(url):
    try:
        response = requests.get(url, timeout=500)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Erreur de connexion à {url} : {e}")
        return pd.DataFrame()

    soup = BeautifulSoup(response.text, 'html.parser')

    job_description_wrappers = soup.find_all('div', class_='job-description-wrapper')

    data_list = []

    for wrapper in job_description_wrappers:
        h5_tag = wrapper.find('h5')
        poste = extract_text(h5_tag, 'a')

        job_recruiter_tag = wrapper.find('p', class_='job-recruiter')
        date_and_company = job_recruiter_tag.text.strip().split('|')
        date = date_and_company[0].strip() if date_and_company else ""
        entreprise = extract_text(job_recruiter_tag, 'a')

        description_tag = wrapper.find('div', class_='search-description')
        description = clean_text(description_tag.text.strip()) if description_tag else ""

        region_tag = wrapper.find('p', text='Région de :')
        region = extract_text(region_tag) if region_tag else ""

        data_list.append({
            'Poste': poste,
            'Entreprise': entreprise,
            'Date': date,
            'Description': description,
            'Région': region
        })

    df = pd.DataFrame(data_list)
    return df

# Liste des liens
categories = ["31", "1127", "29", "37", "1115", "30", "1115", "32", "33", "34", "35", "36", "37", "39", "38", "40", "525", "41", "28"]
#categories=["31"]
# Liste d'URLs générées
urls = ["https://www.emploi.ci/recherche-jobs-cote-ivoire/?f%5B0%5D=im_field_offre_metiers%3A{}".format(category) for category in categories]

# Créer un DataFrame à partir des liens
df = pd.concat([scrape_emploi_ci(url) for url in urls], ignore_index=True)

# Afficher le DataFrame
df



In [ ]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

# Liste des liens
urls = ["https://www.emploi.ci/recherche-jobs-cote-ivoire/?f%5B0%5D=im_field_offre_metiers%3A{}".format(category) for category in categories]


# En-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
options.add_argument("--disable-gpu")  # Désactiver l'accélération GPU en mode headless
chrome_driver_path = "C:\\Users\\Dell\\Desktop\\offre_d_emploi\\chromedriver-win32\\chromedriver.exe"
options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"  # Remplacez par l'emplacement réel de votre Chrome binary
options.add_argument(f"webdriver.chrome.driver={chrome_driver_path}")
driver = webdriver.Chrome(options=options)

# Liste pour stocker les détails de chaque emploi
all_job_details = []

# Parcourir les liens
for url in urls:
    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    offres = soup.find_all('div', class_="job-description-wrapper")

    # Parcourir les offres d'emploi sur la page principale
    for offre in offres:
        # Trouver la balise <h4> dans la structure HTML pour extraire le lien
        offre_link_tag = offre.find('h5')

        # Vérifier si la balise <h4> a été trouvée
        if offre_link_tag:
            # Extraire le lien de l'attribut 'href'
            offre_link = offre_link_tag.find('a')['href']
            all_job_details.append({'Offre_Link': "https://www.emploi.ci"+offre_link})

# Fermer le pilote Selenium à la fin
driver.quit()

# Concaténer tous les détails des emplois en un seul DataFrame
if all_job_details:
    all_job_details_df = pd.DataFrame(all_job_details)
    # Afficher le DataFrame
    #print(all_job_details_df)
else:
    print("Aucun détail d'offre d'emploi trouvé.")


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from requests.exceptions import ChunkedEncodingError, ConnectionError, ReadTimeout

# Fonction pour extraire les informations d'une page
def extract_information(url):
    try:
        response = requests.get(url, timeout=120)  # Augmentation du délai à 20 secondes
        response.raise_for_status()
        response.encoding = 'utf-8'

        soup = BeautifulSoup(response.content, 'html.parser')

        # Extraction des informations sur l'entreprise
        company_info = soup.select_one('.job-ad-company')
        entreprise = {
            'Nom': company_info.select_one('.company-title a').text.strip() if company_info and company_info.select_one('.company-title a') else None,
            'Secteur d´activité': ', '.join(item.text.strip() for item in company_info.select('.sector-title .field-item')) if company_info and company_info.select('.sector-title .field-item') else None,
            'Description de l\'entreprise': soup.select_one('.job-ad-company-description label + *').text.strip() if soup.select_one('.job-ad-company-description label + *') else None
        }

        # Extraction des informations sur l'annonce
        annonce_info = soup.select_one('.job-ad-details')
        annonce = {
            'Poste': soup.select_one('.ad-ss-title').text.strip() if soup.select_one('.ad-ss-title') else None,
            'Missions': [li.text.strip() for li in soup.select('.content ul.missions li')] if soup.select('.content ul.missions') else None,
            'Profil recherché': [li.text.strip() for li in soup.select('.content ul.profil li')] if soup.select('.content ul.profil') else None,
            'Métier': soup.select_one('.job-ad-criteria .field-name-field-offre-metiers .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-metiers .field-item') else None,
            'Secteur d´activité (de l\'annonce)': soup.select_one('.job-ad-criteria .field-name-field-offre-secteur .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-secteur .field-item') else None,
            'Type de contrat': soup.select_one('.job-ad-criteria .field-name-field-offre-contrat-type .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-contrat-type .field-item') else None,
            'Région': soup.select_one('.job-ad-criteria .field-name-field-offre-region .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-region .field-item') else None,
            'Ville': soup.select_one('.job-ad-criteria .field-name-field-offre-ville .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-ville .field-item') else None,
            'Niveau d\'expérience': soup.select_one('.job-ad-criteria .field-name-field-offre-niveau-experience .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-niveau-experience .field-item') else None,
            'Niveau d\'études': soup.select_one('.job-ad-criteria .field-name-field-offre-niveau-etude .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-niveau-etude .field-item') else None,
            'Compétences clés': [li.text.strip() for li in soup.select('.job-ad-criteria .field-name-field-offre-tags .field-item')] if soup.select('.job-ad-criteria .field-name-field-offre-tags .field-item') else None,
            'Nombre de poste(s)': soup.select_one('.job-ad-criteria td:contains("Nombre de poste(s) :") + td').text.strip() if soup.select_one('.job-ad-criteria td:contains("Nombre de poste(s) :") + td') else None,
}

        return {'entreprise': entreprise, 'annonce': annonce}

    except (ConnectionError, ReadTimeout, ChunkedEncodingError) as e:
        print(f"Erreur lors de la requête {url}: {e}")
        # Relancer la requête
        return None

# Liste des URLs
urls = list(all_job_details_df['Offre_Link'])

# Initialisation d'une liste pour stocker les DataFrames
df_list = []

# Boucle à travers chaque URL
for url in urls:
    data = extract_information(url)

    # Si la requête a échoué, passez à l'URL suivante
    if data is None:
        continue

    # Création du DataFrame pour chaque URL
    df = pd.DataFrame([data['entreprise'] | data['annonce']])

    # Ajout du DataFrame à la liste
    df_list.append(df)

# Concaténation des DataFrames de chaque URL
result_df = pd.concat(df_list, ignore_index=True)
result_df


In [ ]:
# Ajouter les listes existantes en tant que colonnes au DataFrame
result_df['Poste'] = list(df['Poste'])
result_df['Entreprise'] = list(df['Entreprise'])
result_df['Date'] =  list(df['Date'])
result_df['Description'] =  list(df['Description'])
result_df['Région'] =  list(df['Région'])

# Réorganiser les colonnes selon vos besoins

result_df

In [ ]:
# Exportez le DataFrame en fichier CSV
result_df.to_csv("df_Novojob.csv", index=False)

# ci.talent

In [ ]:

import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_text(element, tag_name=None):
    tag = element.find(tag_name)
    return tag.text.strip() if tag else ""

def clean_text(text):
    return text.replace('\r\n', '').replace('\xa0', '')

def scrape_talent_com(url):
    try:
        response = requests.get(url, timeout=500)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Erreur de connexion à {url} : {e}")
        return pd.DataFrame()

    soup = BeautifulSoup(response.text, 'html.parser')

    job_wrappers = soup.find_all('div', class_='card card__job')

    data_list = []

    for wrapper in job_wrappers:
        title_tag = wrapper.find('h2', class_='card__job-title')
        title = extract_text(title_tag, 'a')
        
        employer_location_tag = wrapper.find('div', class_='card__job-empnameLocation')
        #employer = extract_text(employer_location_tag.find('div', class_='card__job-location'))  # Extract location from the inner div
        location= extract_text(employer_location_tag, 'div')

        employer_location_tag = wrapper.find('div', class_='card__job-empname-label')
        
        # Extracting employer and description from the div
        employer = employer_location_tag.text.strip() if employer_location_tag else None  # Extract location from the inner div
        
        description_tag = wrapper.find('div', class_='card__job-snippet-logo')
        description = clean_text(extract_text(description_tag, 'p'))

        data_list.append({
            'Title': title,
            'Location': location,
            'Employer': employer,
            'Description': description
        })

    df = pd.DataFrame(data_list)
    return df

# List of URLs for talent.com jobs
urls = [
     "https://ci.talent.com/jobs?l=Abidjan%2C+Abidjan&radius=15&p={}&k=&context=serp_pagination".format(category) for category in range(8)
    # Add more URLs as needed "https://ci.talent.com/jobs",
]

# Initialize an empty DataFrame to store the results
combined_df = pd.DataFrame()

# Scrape job information for each URL and concatenate the results
for url in urls:
    df = scrape_talent_com(url)
    combined_df = pd.concat([combined_df, df], ignore_index=True)

# Display the combined DataFrame
combined_df


In [ ]:
# Exportez le DataFrame en fichier CSV
combined_df.to_csv("d_talent.csv", index=False)

# projobivoire

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

def extract_text(element, tag_name=None):
    if element and tag_name:
        tag = element.find(tag_name)
        return tag.text.strip() if tag else ""
    return ""

def clean_text(text):
    return text.replace('\r\n', '').replace('\xa0', '')

def scrape_projobivoire_page(page_url):
    job_data_list = []

    for url in page_url:
        try:
            response = requests.get(url, timeout=500)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            
            print(f"Erreur de connexion à {url} : {e}")
            continue

        soup = BeautifulSoup(response.text, 'html.parser')

        job_items = soup.find_all('div', class_='loop-item-wrap list')

        if not job_items:
            
            print(f"Aucun élément de travail trouvé pour l'URL : {url}")
            continue

        for job_item in job_items:
            title_tag = job_item.find('h3', class_='loop-item-title')
            title = extract_text(title_tag, 'a')

            job_type_tag = job_item.find('span', class_='job-type')
            job_type = extract_text(job_type_tag, 'span')

            job_date_posted = soup.find('span', class_='job-date__posted').text.strip()

            job_date_closing_tag = soup.find('span', class_='job-date__closing')
            job_date_closing = job_date_closing_tag.text.strip() if job_date_closing_tag else ""

            job_date_closing = job_date_closing.lstrip('-').strip()

            category_tag = job_item.find('span', class_='job-category')
            category = extract_text(category_tag, 'a')

            # Ajout de ces lignes pour extraire l'URL de l'e-mail
            email_url_tag = job_item.find('span', class_='noo-tool-email-job')
            email_url = email_url_tag['data-url'] if email_url_tag else ""

            data = {
                'Title': title,
                'Type': job_type,
                'DatePosted': job_date_posted,
                'DateClosing': job_date_closing,
                'Category': category,
                'EmailURL': email_url,
            }

            job_data_list.append(data)

    return job_data_list

# Liste des URL de pages avec plusieurs offres d'emploi
page_urls = ["https://projobivoire.com/page/{}/".format(category) for category in range(546)]

# Scrape des détails de chaque offre d'emploi sur les pages
job_data_list = scrape_projobivoire_page(page_urls)

# Création d'un DataFrame à partir de la liste des données d'emploi
df_projobivoire = pd.DataFrame(job_data_list)

# Affichage du DataFrame
df_projobivoire

In [ ]:
df_projobivoire["EmailURL"][0]

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_job_information(soup):
    # Extraction des informations de l'offre d'emploi
    poste = soup.select_one('li.list-group-item:-soup-contains("Poste")').strong.next_sibling.strip()
    type_offre = soup.select_one('li.list-group-item:-soup-contains("Type d\'offre")').strong.next_sibling.strip()
    metiers = soup.select_one('li.list-group-item:-soup-contains("Métier(s):")').strong.next_sibling.strip()
    niveaux = soup.select_one('li.list-group-item:-soup-contains("Niveau(x):")').strong.next_sibling.strip()
    experience = soup.select_one('li.list-group-item:-soup-contains("Expérience:")').strong.next_sibling.strip()
    lieu = soup.select_one('li.list-group-item:-soup-contains("Lieu:")').strong.next_sibling.strip()
    
    # Extraction des dates de publication et de limite
    date_publication = soup.find('strong', string='Date de publication:').find_next('span').text.strip()
    date_limite = soup.find('strong', string='Date limite:').find_next('span').text.strip()
    
   # description = soup.select_one('div.text-col.post.small-post.col-md-9.col-xs-12 ul.list-group').text.strip()
    description = soup.select_one('div.entry-content').text.strip()

    return {
        "Poste": [poste],
        "Type d'offre": [type_offre],
        "Métier(s)": [metiers],
        "Niveau(x)": [niveaux],
        "Expérience": [experience],
        "Lieu": [lieu],
        "Date de publication": [date_publication],
        "Date limite": [date_limite],
        "Description": [description]
    }

# Liste des URLs à scraper
urls = list(df_projobivoire['EmailURL'])

# Liste pour stocker les DataFrames
dfs = []

# Boucle sur chaque URL
for url in urls:
    # Envoyer une requête GET au site
    response = requests.get(url)

    # Vérifier si la requête a réussi (statut 200)
    if response.status_code == 200:
        # Analyser le contenu de la page avec BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        try:
            # Extract job information
            job_info = extract_job_information(soup)

            # Create DataFrame
            df = pd.DataFrame(job_info)

            # Ajouter le DataFrame à la liste
            dfs.append(df)
        except Exception as e:
            print(f"An error occurred: {e}")
    else:
        print(f"Échec de la requête pour l'URL {url}. Statut : {response.status_code}")

# Concaténer tous les DataFrames en un seul DataFrame
df_E = pd.concat(dfs, ignore_index=True)
df_E


In [ ]:
# Ajouter les listes existantes en tant que colonnes au DataFrame
df_E['Title'] = list(df_projobivoire['Title'])
df_E['Type'] = list(df_projobivoire['Type'])
df_E['DatePosted'] =  list(df_projobivoire['DatePosted'])
df_E['DateClosing'] =  list(df_projobivoire['DateClosing'])
df_E['Category'] =  list(df_projobivoire['Category'])
df_E['EmailURL'] =  list(df_projobivoire['EmailURL'])

# Réorganiser les colonnes selon vos besoins

df_E

In [ ]:
# Exportez le DataFrame en fichier CSV
df_projobivoire.to_csv("df_projobivoire.csv", index=False)

# Emploi_jeune.ci

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# List of URLs to scrape
urls = [
    "https://agenceemploijeunes.ci/site/offres-emplois?page={}".format(category) for category in range(17)
    # Add more URLs as needed
]

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Lists to store data
job_titles = []
publication_dates = []
application_deadlines = []
locations = []
job_descriptions = []
job_types = []
diploma_requirements = []

# Loop through each URL
for url in urls:
    # Send a request to the website
    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')

    # Find job listings
    job_listings = soup.find_all('div', class_='post-bx')

    # Extract data from each job listing
    for job_listing in job_listings:
    # Job title
        job_title = job_listing.find('h4').text.strip()
        job_titles.append(job_title)

    # Publication date and application deadline
        date_info = job_listing.find_all('li', {'class': ''})
        if date_info:
            publication_date = date_info[0].text.replace('Publié le:', '').strip()
            application_deadline = date_info[1].text.replace('Date limite:', '').strip()
            publication_dates.append(publication_date)
            application_deadlines.append(application_deadline)

    # Location
        location = date_info[2].text.replace('ABENGOUROU', '').strip()
        locations.append(location)

    # Job description
        job_description = job_listing.find('p').text.strip()
        job_descriptions.append(job_description)

    # Job type
        job_type = job_listing.find('span', {'class': 'pull-right'}).text.strip()
        job_types.append(job_type)

    # Diploma requirement
        diploma_requirement = job_listing.find('div', {'class': 'salary-bx'}).text.strip()
        diploma_requirements.append(diploma_requirement)


# Create a Pandas DataFrame
data = {
    'Job Title': job_titles,
    'Publication Date': publication_dates,
    'Application Deadline': application_deadlines,
    'Location': locations,
    'Job Description': job_descriptions,
    'Job Type': job_types,
    'Diploma Requirement': diploma_requirements
}

df_agenceemploijeunes = pd.DataFrame(data)
# Utiliser la méthode str.extract pour extraire la valeur après "Diplôme :"
df_agenceemploijeunes['Diplome'] = df_agenceemploijeunes['Diploma Requirement'].str.extract(r'Diplôme :[ \t]*([^\n\r]*)')


# Print or further process the DataFrame
df_agenceemploijeunes




In [ ]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

# Liste des liens

# En-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
options.add_argument("--disable-gpu")  # Désactiver l'accélération GPU en mode headless
chrome_driver_path = "C:\\Users\\ngora\\OneDrive\\Bureau\\INS_DATA\\chromedriver_win32\\chromedriver.exe"
options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"  # Remplacez par l'emplacement réel de votre Chrome binary
options.add_argument(f"webdriver.chrome.driver={chrome_driver_path}")
driver = webdriver.Chrome(options=options)

# Liste pour stocker les détails de chaque emploi
all_job_details = []

# Parcourir les liens
for url in urls:
    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    offres = soup.find_all('div', class_="col-xl-9 col-lg-8 col-md-7")

    # Parcourir les offres d'emploi sur la page principale
    for offre in offres:
        # Trouver la balise <h4> dans la structure HTML pour extraire le lien
        offre_link_tag = offre.find('h4')

        # Vérifier si la balise <h4> a été trouvée
        if offre_link_tag:
            # Extraire le lien de l'attribut 'href'
            offre_link = offre_link_tag.find('a')['href']
            all_job_details.append({'Offre_Link': offre_link})

# Fermer le pilote Selenium à la fin
driver.quit()

# Concaténer tous les détails des emplois en un seul DataFrame
if all_job_details:
    all_job_details_df = pd.DataFrame(all_job_details)
    # Afficher le DataFrame
    #print(all_job_details_df)
else:
    print("Aucun détail d'offre d'emploi trouvé.")
    
all_job_details_df    


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# List of URLs to scrape
urls = list(all_job_details_df["Offre_Link"])
# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Lists to store data
job_titles = []
locations = []
references = []
number_of_positions = []
closing_dates = []
diplomas = []
job_types = []
experiences = []
education_levels = []
genders = []
job_descriptions = []

# Loop through each URL
for url in urls:
    # Send a request to the website
    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    offre_url = url

    # Extract job details
    job_details = soup.find('div', class_='widget_getintuch')

    if job_details:
        # Extract data from job details
        ul_element = job_details.find('ul')
        if ul_element:
            details_list = ul_element.find_all('li')

            # Iterate through details
            for detail in details_list:
                label = detail.find('strong')
                value_span = detail.find('span', class_='text-black-light')

                if label and value_span:
                    label_text = label.text.strip()
                    value_text = value_span.text.strip()

                    if 'Lieu de travail' in label_text:
                        locations.append(value_text)
                    elif 'Reference' in label_text:
                        references.append(value_text)
                    elif 'Nombre de poste' in label_text:
                        number_of_positions.append(value_text)
                    elif 'Date de clôture' in label_text:
                        closing_dates.append(value_text)
                    elif 'Diplôme' in label_text:
                        diplomas.append(value_text)
                    elif 'Type de contrat' in label_text:
                        job_types.append(value_text)
                    elif 'Expérience professionnelle' in label_text:
                        experiences.append(value_text)
                    elif 'Niveau d\'études' in label_text:
                        education_levels.append(value_text)
                    elif 'Sexe' in label_text:
                        genders.append(value_text)

        # Extract job title and description
        job_title_element = soup.find('h3', {'class': 'title-head'})
        if job_title_element:
            job_title = job_title_element.text.strip()
            job_titles.append(job_title)

            job_description_info = soup.find('div', {'class': 'job-info-box'}).find('ul')
            if job_description_info:
                job_description_text = '\n'.join([li.text.strip() for li in job_description_info.find_all('li')])
                job_descriptions.append(job_description_text)
            else:
                job_descriptions.append(None)
        else:
            job_titles.append(None)
            job_descriptions.append(None)
    else:
        job_titles.append(None)
        job_descriptions.append(None)

# Create a Pandas DataFrame
data = {
    'offre_url':offre_url,
    'Job Title': job_titles,
    'Location': locations,
    'Reference': references,
    'Number of Positions': number_of_positions,
    'Closing Date': closing_dates,
    'Diploma': diplomas,
    'Job Type': job_types,
    'Experience': experiences,
    'Education Level': education_levels,
    'Gender': genders,
    'Job Description': job_descriptions
}

df_job_details = pd.DataFrame(data)

# Print or further process the DataFrame
df_job_details


In [ ]:
# Ajouter les listes existantes en tant que colonnes au DataFrame
df_job_details['Job Title'] = list(df_agenceemploijeunes['Job Title'])
df_job_details['Publication Date'] = list(df_agenceemploijeunes['Type'])
df_job_details['Application Deadline'] =  list(df_agenceemploijeunes['Application Deadline'])
df_job_details[ 'Location'] =  list(df_agenceemploijeunes[ 'Location'])
df_job_details['Job Description'] =  list(df_agenceemploijeunes['Job Description'])
df_job_details['Job Type'] =  list(df_agenceemploijeunes['Job Type'])
df_job_details['Diploma Requirement'] =  list(df_agenceemploijeunes['Diploma Requirement'])

# Réorganiser les colonnes selon vos besoins


df_job_details

In [ ]:
# Exportez le DataFrame en fichier CSV
df_job_details.to_csv("df_agenceemploijeunes.csv", index=False)

# Alerteemploi.net

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Liste des URLs des offres d'emploi
job_listing_urls = [
    "https://alerteemploi.net/toutes-les-offres/",
    # Ajoutez d'autres URLs au besoin
]

# Configurez Selenium pour s'exécuter en mode headless (sans ouvrir de fenêtre de navigateur)
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)

# Liste pour stocker les données des offres d'emploi
job_data = []

# Boucle à travers chaque URL d'offre d'emploi
for url in job_listing_urls:
    # Envoyez une requête GET en utilisant Selenium
    driver.get(url)

    while True:
        try:
            # Cliquez sur le bouton "Load more listings"
            load_more_button = driver.find_element(By.CLASS_NAME, 'load_more_jobs')
            load_more_button.click()

            # Attendez le chargement des nouvelles offres d'emploi
            WebDriverWait(driver, 10).until(
                EC.invisibility_of_element_located((By.CLASS_NAME, 'loading_jobs'))
            )

            # Obtenez le code source de la page mis à jour
            page_source = driver.page_source

            # Utilisez BeautifulSoup pour analyser le HTML
            soup = BeautifulSoup(page_source, 'html.parser')

            # Trouvez toutes les offres d'emploi sur la page
            job_listings = soup.find_all('li', class_='job_listing')

            # Boucle à travers chaque offre d'emploi et extrayez les informations
            for listing in job_listings:
                job_title = listing.find('div', class_='position').find('h3').text.strip()
                company = listing.find('div', class_='company').find('strong').text.strip()
                location = listing.find('div', class_='location').text.strip()
                date_posted_element = listing.find('li', class_='date').find('time')
                date_posted = date_posted_element['datetime'].strip() if date_posted_element else None
                listing_url = listing.find('a')['href']

                # Stockez les données dans un dictionnaire
                job_entry = {
                    'Titre du poste': job_title,
                    'Entreprise': company,
                    'Lieu': location,
                    'Date de publication': date_posted,
                    'URL': listing_url
                }

                job_data.append(job_entry)

        except Exception as e:
            # Sortez de la boucle si le bouton n'est pas trouvé ou s'il y a une exception
            break

# Fermez le pilote Selenium
driver.quit()

# Créez un DataFrame
df_alerteemploi = pd.DataFrame(job_data)

# Affichez le DataFrame
df_alerteemploi


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

# Liste d'URLs des pages d'offres d'emploi
urls = list(df_alerteemploi["URL"])
# Initialiser une liste pour stocker les données
job_data = []

# Configurer Selenium pour s'exécuter en mode headless (sans ouvrir de fenêtre de navigateur)
chrome_options = Options()
chrome_options.add_argument("--headless")

# Boucle à travers chaque URL
for url in urls:
    try:
        # Initialiser le pilote Selenium
        driver = webdriver.Chrome(options=chrome_options)

        # Envoyer une requête GET en utilisant Selenium
        driver.get(url)

        # Attendre quelques secondes (ajustez selon les besoins)
        time.sleep(30)

        # Récupérer le code source de la page après l'exécution de JavaScript
        page_source = driver.page_source

        # Fermer le pilote Selenium
        driver.quit()

        # Utiliser BeautifulSoup pour analyser le HTML
        soup = BeautifulSoup(page_source, 'html.parser')

        # Extraire les détails de l'offre d'emploi
        job_title_element = soup.find('h1', class_='entry-title')
        job_title = job_title_element.text.strip() if job_title_element else None

        # Check if the initial element is found before attempting to find the nested element
        author_container = soup.find('div', class_='td-post-author-name')
        author_element = author_container.find('a') if author_container else None
        author = author_element.text.strip() if author_element else None

        date_posted_element = soup.find('time', class_='entry-date')
        date_posted = date_posted_element['datetime'].strip() if date_posted_element else None

        views_element = soup.find('div', class_='td-post-views')
        views = views_element.find('span', class_='td-nr-views-19100').text.strip() if views_element and views_element.find('span', class_='td-nr-views-19100') else None

        # Check if the element is found before accessing its properties
        image_url_element = soup.find('div', class_='td-post-featured-image')
        image_url = image_url_element.find('img')['src'] if image_url_element and image_url_element.find('img') else None

        # Ajouter les détails à la liste
        job_data.append({
            'Job Title': job_title,
            'Author': author,
            'Date Posted': date_posted,
            'Views': views,
            'Image URL': image_url,
            'Link URL': url  # Ajoutez l'URL de la page à la liste
        })
    except Exception as e:
        print(f"Une erreur s'est produite pour l'URL {url}: {str(e)}")

# Créer un DataFrame avec les données extraites

df = pd.DataFrame(job_data)

# Afficher le DataFrame
df


In [ ]:
# Ajouter les listes existantes en tant que colonnes au DataFrame
df['Titre du poste'] = list(df_alerteemploi['Titre du poste'])
df['Entreprise'] = list(df_alerteemploi['Entreprise'])
df['Lieu'] =  list(df_alerteemploi['Lieu'])
df[ 'Date de publication'] =  list(df_alerteemploi['Date de publication'])
df['URL'] =  list(df_alerteemploi['URL'])

# Réorganiser les colonnes selon vos besoins




df_job_details

In [ ]:
# Exportez le DataFrame en fichier CSV
df.to_csv("df_alerteemploi.csv", index=False)

# jobcenter

In [ ]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options

# Liste des URLs des pages d'offres d'emploi
urls = [
    "https://rmo-jobcenter.com/fr/nos-offres-emploi.html",
    # Ajoutez d'autres URLs au besoin
]

# Configurez Selenium pour s'exécuter en mode headless
chrome_options = Options()
chrome_options.add_argument("--headless")

# Liste pour stocker les données des offres d'emploi
all_job_data = []

# Base URL of the website
base_url = "https://rmo-jobcenter.com"

# Boucle à travers chaque URL
for url in urls:
    # Initialisez le pilote Selenium
    driver = webdriver.Chrome(options=chrome_options)

    # Chargez la page avec Selenium
    driver.get(url)

    # Récupérez le code source de la page après l'exécution du JavaScript
    page_source = driver.page_source

    # Fermez le pilote Selenium
    driver.quit()

    # Utilisez BeautifulSoup pour analyser le HTML
    soup = BeautifulSoup(page_source, 'html.parser')

    # Trouvez la table contenant les offres d'emploi
    table = soup.find('table', class_='liste')

    # Liste pour stocker les données des offres d'emploi pour une URL spécifique
    job_data = []

    # Boucle à travers chaque ligne de la table (sauf la première qui contient les en-têtes)
    for row in table.find_all('tr')[1:]:
        # Extrayez les données de chaque colonne
        columns = row.find_all('td')
        date = columns[0].text.strip()
        filiale = columns[1].text.strip()
        fonction = columns[2].text.strip()
        secteur = columns[3].text.strip()
        reference = columns[4].text.strip()
        details_url = columns[5].find('a')['href']

        # Rendez l'URL absolue en la combinant avec l'URL de base.
        absolute_url = f"{base_url}/{details_url}"

        # Stockez les données dans un dictionnaire
        job_entry = {
            'Date': date,
            'Filiale': filiale,
            'Fonction': fonction,
            'Secteur': secteur,
            'Référence / Statut': reference,
            'Détails URL': absolute_url
        }

        job_data.append(job_entry)

    # Ajoutez les données de cette URL à la liste globale
    all_job_data.extend(job_data)

# Créez un DataFrame avec toutes les données extraites
df_jobcenter = pd.DataFrame(all_job_data)

# Affichez le DataFrame
df_jobcenter


In [ ]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options

# Liste des URLs des pages d'offres d'emploi
urls = list(df_jobcenter['Détails URL'])

# Configurez Selenium pour s'exécuter en mode headless
chrome_options = Options()
chrome_options.add_argument("--headless")

# Liste pour stocker les données des offres d'emploi
all_job_data = []

# Boucle à travers chaque URL
for url in urls:
    # Initialisez le pilote Selenium
    driver = webdriver.Chrome(options=chrome_options)

    # Chargez la page avec Selenium
    driver.get(url)

    # Récupérez le code source de la page après l'exécution du JavaScript
    page_source = driver.page_source

    # Fermez le pilote Selenium
    driver.quit()

    # Utilisez BeautifulSoup pour analyser le HTML
    soup = BeautifulSoup(page_source, 'html.parser')

    # Trouvez la div contenant les informations détaillées
    details_div = soup.find('div', {'id': 'content_articles'})

    # Extract details from the div
    job_title = details_div.find('div', {'id': 'h2_imprime'}).text.strip()
    job_description = details_div.find('div', {'class': 'text-content'}).text.strip()

    # Additional details can be extracted similarly

    # Stockez les données dans un dictionnaire
    job_entry = {
        'Job Title': job_title,
        'Job Description': job_description,
        'Job URL': url
        # Add more details as needed
    }

    # Ajoutez les données de cette URL à la liste globale
    all_job_data.append(job_entry)

# Créez un DataFrame avec toutes les données extraites
df_details = pd.DataFrame(all_job_data)

# Affichez le DataFrame
df_details


In [ ]:
# Ajouter les listes existantes en tant que colonnes au DataFrame
df_jobcenter['Job Title'] = list(df_details['Job Title'])
df_jobcenter['Job Description'] = list(df_details['Job Description'])
df_jobcenter['Job URL'] =  list(df_details['Job URL'])

# Réorganiser les colonnes selon vos besoins




df_job_details

In [ ]:
# Exportez le DataFrame en fichier CSV
df_jobcenter.to_csv("df_jobcenter.csv", index=False)

# yop.l-frii

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

# List of URLs
urls = [
    "https://yop.l-frii.com/offres-demplois/{}/".format(category) for category in range(3276)
    # Add more URLs as needed
]

job_data = []

for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    job_articles = soup.find_all('article', class_='type-emploi')

    for article in job_articles:
        # Check if the 'h2' element with class 'elementor-heading-title' is found
        job_title_element = article.find('h2', class_='elementor-heading-title')
        job_title = job_title_element.text.strip() if job_title_element else None

        job_link = article.find('a', href=True)['href']
        
        # Check if the image element is found
        job_image_element = article.find('img', class_='attachment-large')
        job_image = job_image_element['src'] if job_image_element else None

        job_data.append({
            "Job Title": job_title,
            "Job Link": job_link,
            "Job Image": job_image,
            "Source URL": url  # Include the source URL in the DataFrame
        })

df_yop_l_frii = pd.DataFrame(job_data)
df_yop_l_frii


In [ ]:
# Exportez le DataFrame en fichier CSV
df_yop_l_frii.to_csv("df_yop_l_frii.csv", index=False)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

from selenium import webdriver

def emploi_ci():
    def extract_text(element, tag_name=None):
        tag = element.find(tag_name)
        return tag.text.strip() if tag else ""

    def clean_text(text):
        return text.replace('D\x92', ' ').replace('d\x92', ' ').replace('\x92', ' ').replace('\r\n', '').replace('\xa0', '')

    def scrape_emploi_ci(url):
        try:
            response = requests.get(url, timeout=500)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Erreur de connexion à {url} : {e}")
            return pd.DataFrame()

        soup = BeautifulSoup(response.text, 'html.parser')

        job_description_wrappers = soup.find_all('div', class_='job-description-wrapper')

        data_list = []

        for wrapper in job_description_wrappers:
            h5_tag = wrapper.find('h5')
            poste = extract_text(h5_tag, 'a')

            job_recruiter_tag = wrapper.find('p', class_='job-recruiter')
            date_and_company = job_recruiter_tag.text.strip().split('|')
            date = date_and_company[0].strip() if date_and_company else ""
            entreprise = extract_text(job_recruiter_tag, 'a')

            description_tag = wrapper.find('div', class_='search-description')
            description = clean_text(description_tag.text.strip()) if description_tag else ""

            region_tag = wrapper.find('p', text='Région de :')
            region = extract_text(region_tag) if region_tag else ""

            data_list.append({
                'Poste': poste,
                'Entreprise': entreprise,
                'Date': date,
                'Description': description,
                'Région': region,
                'URL' : url
            })

        df = pd.DataFrame(data_list)
        return df

    # Liste des liens
    categories = ["31", "1127", "29", "37", "1115", "30", "1115", "32", "33", "34", "35", "36", "37", "39", "38", "40", "525", "41", "28"]
    #categories=["31"]
    # Liste d'URLs générées
    urls = ["https://www.emploi.ci/recherche-jobs-cote-ivoire/?f%5B0%5D=im_field_offre_metiers%3A{}".format(category) for category in categories]

    # Créer un DataFrame à partir des liens
    df = pd.concat([scrape_emploi_ci(url) for url in urls], ignore_index=True)


    from requests.exceptions import ChunkedEncodingError, ConnectionError, ReadTimeout

    # Liste des liens

    # En-tête pour éviter d'être bloqué
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
    options.add_argument("--disable-gpu")  # Désactiver l'accélération GPU en mode headless
    chrome_driver_path = "C:\\Users\\ngora\\OneDrive\\Bureau\\INS_DATA\\chromedriver_win32\\chromedriver.exe"
    options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"  # Remplacez par l'emplacement réel de votre Chrome binary
    options.add_argument(f"webdriver.chrome.driver={chrome_driver_path}")
    driver = webdriver.Chrome(options=options)

    # Liste pour stocker les détails de chaque emploi
    all_job_details = []

    # Parcourir les liens
    for url in list(df["URL"]):
        req = requests.get(url, headers=headers)
        soup = BeautifulSoup(req.text, 'html.parser')
        time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

        offres = soup.find_all('div', class_="job-description-wrapper")

        # Parcourir les offres d'emploi sur la page principale
        for offre in offres:
            # Trouver la balise <h4> dans la structure HTML pour extraire le lien
            offre_link_tag = offre.find('h5')

            # Vérifier si la balise <h4> a été trouvée
            if offre_link_tag:
                # Extraire le lien de l'attribut 'href'
                offre_link = offre_link_tag.find('a')['href']
                all_job_details.append({'Offre_Link': "https://www.emploi.ci"+offre_link, 'URL' :url})

    # Fermer le pilote Selenium à la fin
    driver.quit()

    # Concaténer tous les détails des emplois en un seul DataFrame
    if all_job_details:
        all_job_details_df = pd.DataFrame(all_job_details)
        # Afficher le DataFrame
        #print(all_job_details_df)
    else:
        print("Aucun détail d'offre d'emploi trouvé.")

    # Fusionner les deux DataFrames sur la colonne 'URL'
    emploi_df = pd.merge(df, all_job_details_df, on='URL')
    emploi_df = emploi_df.drop_duplicates()
    
    #df["URL"]=list(all_job_details_df["URL"])
    #df["Offre_Link"]=list(all_job_details_df["Offre_Link"])

    # Fonction pour extraire les informations d'une page
    def extract_information(url):
        try:
            response = requests.get(url, timeout=120)  # Augmentation du délai à 20 secondes
            response.raise_for_status()
            response.encoding = 'utf-8'

            soup = BeautifulSoup(response.content, 'html.parser')

            # Extraction des informations sur l'entreprise
            company_info = soup.select_one('.job-ad-company')
            entreprise = {
                "Offre_Link" : url,
                'Nom': company_info.select_one('.company-title a').text.strip() if company_info and company_info.select_one('.company-title a') else None,
                'Secteur d´activité': ', '.join(item.text.strip() for item in company_info.select('.sector-title .field-item')) if company_info and company_info.select('.sector-title .field-item') else None,
                'Description de l\'entreprise': soup.select_one('.job-ad-company-description label + *').text.strip() if soup.select_one('.job-ad-company-description label + *') else None
            }

            # Extraction des informations sur l'annonce
            annonce_info = soup.select_one('.job-ad-details')
            annonce = {
                'Poste': soup.select_one('.ad-ss-title').text.strip() if soup.select_one('.ad-ss-title') else None,
                'Missions': [li.text.strip() for li in soup.select('.content ul.missions li')] if soup.select('.content ul.missions') else None,
                'Profil recherché': [li.text.strip() for li in soup.select('.content ul.profil li')] if soup.select('.content ul.profil') else None,
                'Métier': soup.select_one('.job-ad-criteria .field-name-field-offre-metiers .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-metiers .field-item') else None,
                'Secteur d´activité (de l\'annonce)': soup.select_one('.job-ad-criteria .field-name-field-offre-secteur .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-secteur .field-item') else None,
                'Type de contrat': soup.select_one('.job-ad-criteria .field-name-field-offre-contrat-type .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-contrat-type .field-item') else None,
                'Région': soup.select_one('.job-ad-criteria .field-name-field-offre-region .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-region .field-item') else None,
                'Ville': soup.select_one('.job-ad-criteria .field-name-field-offre-ville .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-ville .field-item') else None,
                'Niveau d\'expérience': soup.select_one('.job-ad-criteria .field-name-field-offre-niveau-experience .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-niveau-experience .field-item') else None,
                'Niveau d\'études': soup.select_one('.job-ad-criteria .field-name-field-offre-niveau-etude .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-niveau-etude .field-item') else None,
                'Compétences clés': [li.text.strip() for li in soup.select('.job-ad-criteria .field-name-field-offre-tags .field-item')] if soup.select('.job-ad-criteria .field-name-field-offre-tags .field-item') else None,
                'Nombre de poste(s)': soup.select_one('.job-ad-criteria td:contains("Nombre de poste(s) :") + td').text.strip() if soup.select_one('.job-ad-criteria td:contains("Nombre de poste(s) :") + td') else None,
            }

            return {'entreprise': entreprise, 'annonce': annonce}

        except (ConnectionError, ReadTimeout, ChunkedEncodingError) as e:
            print(f"Erreur lors de la requête {url}: {e}")
            # Relancer la requête
            entreprise = {
                "Offre_Link" : url,
                'Nom': "",
                'Secteur d´activité': "",
                'Description de l\'entreprise':""}
            annonce = {'Poste':"",
                       'Missions': "",
                       'Profil recherché':"",
                       'Métier':"",
                       'Secteur d´activité (de l\'annonce)':"",
                       'Type de contrat':"",
                       'Région': "",
                       'Ville':"",
                       'Niveau d\'expérience':"",
                       'Niveau d\'études':"",
                       'Compétences clés':"",
                       'Nombre de poste(s)':""}


            return {'entreprise': entreprise, 'annonce': annonce}

    # Liste des URLs
    urls = list(emploi_df['Offre_Link'])

    # Initialisation d'une liste pour stocker les DataFrames
    df_list = []

    # Boucle à travers chaque URL
    for url in urls:
        data = extract_information(url)

        # Si la requête a échoué, passez à l'URL suivante
        if data is None:
            continue

        # Création du DataFrame pour chaque URL
        df = pd.DataFrame([data['entreprise'] | data['annonce']])

        # Ajout du DataFrame à la liste
        df_list.append(df)

    # Concaténation des DataFrames de chaque URL
    result_df = pd.concat(df_list, ignore_index=True)
    # Ajouter les listes existantes en tant que colonnes au DataFrame
    result_df = pd.merge(result_df, emploi_df, on='Offre_Link')
    result_df = emploi_df.drop_duplicates()


# Réorganiser les colonnes selon vos besoins
#Poste 	Entreprise 	Date 	Description 	Région 	URL 	Offre_Link

    return result_df

# Appel de la fonction pour obtenir le DataFrame
emploi_df=emploi_ci()
emploi_df.reset_index(drop=True, inplace=True)
emploi_df